In [1]:
! pip install  langchain openai pymysql --upgrade -q


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
! pip install langchain_experimental

     ------------------------------------ 177.6/177.6 kB 824.6 kB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.agents import initialize_agent
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper

In [2]:
os.environ['OPENAI_API_KEY'] = "enter key"
llm = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo'# Here you can use gpt 4 model
)

C:\Users\asus\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:
host = 'localhost'
port = '3306'
username = 'root'
password = "put your password"
database_schema = 'school'
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

db = SQLDatabase.from_uri(mysql_uri, include_tables=['student'], sample_rows_in_table_info=2)

db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [4]:
def data_from_db(query):
    db_context = db_chain(query)
    db_context = db_context['result'].strip()
    return db_context

### Open AI Rag model

In [5]:
def llm_generate(query):
    db_context = data_from_db(query)
    system_message = """You are a Working professional working in a school.
            You have to answer user's queries and provide relevant information to help students aname and their school search. 
            Example:

            Input:
            Name of all those students who studied in AITR?

            Context:
            The name of students who studied in AITR:
            1. Praveen

            Output:
            The name of students who studied in AITR IS PRAVEEN
            
            And if the no record found in sql table than give the output:
            Data Not Available
            """
    
    human_query = HumanMessagePromptTemplate.from_template(
      """Input:
      {human_input}
      
      Context:
      {db_context}
      
      Output:
      """
    )
    messages = [
      SystemMessage(content=system_message),
      human_query.format(human_input=query, db_context=db_context)
    ]
    response = llm(messages).content
    return response

In [6]:
result = llm_generate("name of students who are in kkr")

C:\Users\asus\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SQLDatabaseChain chain...
name of students who are in kkr
SQLQuery:SELECT `NAME` 
FROM student
WHERE `SCHOOL_NAME` = 'KKR'
LIMIT 5;
SQLResult: 
Answer:Final answer here: No students are in KKR.
> Finished chain.


C:\Users\asus\AppData\Local\Programs\Python\Python310\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


### Send the email to the client

In [7]:
if result == "Data Not Available":   
    os.environ["ZAPIER_NLA_API_KEY"] = 'enter key'
    zapier = ZapierNLAWrapper()
    toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
    agent = initialize_agent(toolkit.get_tools(), llm, agent="zero-shot-react-description", verbose=True)
    agent.run("send the email to this address abcd.gmail.com here is the google forms link https://docs.google.com/forms/.... please refer the form and fill the missing informtaion so that i can merge that information in the datsbase ")
else:
    result

'Data Not Available'